### Importaciones

In [4]:
import pandas as pd
from datetime import datetime
import pandas as pd
import numpy as np
import random
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 
Mes -> 	Fecha del mes en formato YYYY-MM-DD. Representa el mes al que corresponden los datos climáticos.
temperature_2m_max	-> Temperatura máxima promedio del mes medida a 2 metros de altura, en grados Celsius (°C).
temperature_2m_min -> Temperatura mínima promedio del mes, también a 2 metros de altura, en grados Celsius (°C).
precipitation_sum -> Precipitación total del mes en milímetros (mm). Suma de toda el agua caída durante ese mes.
winddirection_10m_dominant -> Dirección dominante del viento a 10 metros de altura, en grados. Se mide en sentido azimutal desde el norte. (Ej.: 0° = norte, 90° = este, etc.).
SEREMI -> Nombre de la región sanitaria (SEREMI) a la que corresponde esta fila.


### Creación de DataSet's de clima mensual por año y SEREMI:

In [5]:
# 1. Cargar todos los archivos seremi
seremi_data = {
    "2014": pd.read_csv("seremi_2014.csv"),
    "2015": pd.read_csv("seremi_2015.csv"),
    "2016": pd.read_csv("seremi_2016.csv"),
    "2017": pd.read_csv("seremi_2017.csv"),
    "2018": pd.read_csv("seremi_2018.csv"),
    "2019": pd.read_csv("seremi_2019.csv"),
    "2020": pd.read_csv("seremi_2020.csv"),
    "2021": pd.read_csv("seremi_2021.csv"),
    "2022": pd.read_csv("seremi_2022.csv"),
    "2023": pd.read_csv("seremi_2023.csv"),
    "2024": pd.read_csv("seremi_2024.csv"),
    "2025": pd.read_csv("seremi_2025.csv"),
}

# 2. Función para procesar cada dataset climático
def procesar_datasets_climaticos(seremi_data_dict):
    def grados_a_cardinal(grados):
        if pd.isna(grados):
            return None
        direcciones = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        index = int((grados + 22.5) % 360 // 45)
        return direcciones[index]

    for año, df in seremi_data_dict.items():
        # Eliminar columna innecesaria
        if "Unnamed: 0" in df.columns:
            df = df.drop(columns=["Unnamed: 0"])

        # Crear columna cardinal
        if "winddirection_10m_dominant" in df.columns:
            df["wind_direction_cardinal"] = df["winddirection_10m_dominant"].apply(grados_a_cardinal)

        # Renombrar columnas
        df = df.rename(columns={
            "temperature_2m_max": "Temperatura Máxima",
            "temperature_2m_min": "Temperatura Mínima",
            "precipitation_sum": "Precipitaciones (suma)",
            "winddirection_10m_dominant": "Dirección Viento"
        })

        # Limpiar columna de grados si existe y renombrar cardinal
        if "Dirección Viento" in df.columns:
            df = df.drop(columns=["Dirección Viento"])
        if "wind_direction_cardinal" in df.columns:
            df = df.rename(columns={"wind_direction_cardinal": "Dirección del Viento"})

        # Guardar el dataframe procesado
        seremi_data_dict[año] = df

    return seremi_data_dict

# 3. Aplicar la función a todos los datasets climáticos
seremi_data = procesar_datasets_climaticos(seremi_data)

# 4. Verificar que todo esté correcto (opcional)
seremi_data["2015"].head(12)

,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),SEREMI,Dirección del Viento
0,2015-01-01,12.277419,6.867742,33.7,Magallanes,W
1,2015-02-01,12.142857,7.100000,36.9,Magallanes,W
2,2015-03-01,11.561290,6.393548,92.1,Magallanes,W
3,2015-04-01,9.093333,4.446667,57.9,Magallanes,W
4,2015-05-01,5.935484,2.541935,100.0,Magallanes,S
5,2015-06-01,3.943333,1.150000,92.8,Magallanes,SW
6,2015-07-01,3.032258,-0.138710,98.4,Magallanes,SW
7,2015-08-01,4.180645,0.509677,59.8,Magallanes,SW
8,2015-09-01,5.266667,1.120000,46.0,Magallanes,SW
9,2015-10-01,8.925806,2.803226,24.2,Magallanes,SW


### Creación de DataSet's sobre ocupación de camas por mes y por año según hospital y área:

In [21]:
# Archivos filtrados hospitalarios
archivos_filtrados = {
    "2014": "2014_filtrado.csv",
    "2015": "2015_filtrado.csv",
    "2016": "2016_filtrado.csv",
    "2017": "2017_filtrado.csv",
    "2018": "2018_filtrado.csv",
    "2019": "2019_filtrado.csv",
    "2020": "2020_filtrado.csv",
    "2021": "2021_filtrado.csv",
    "2022": "2022_filtrado.csv",
    "2023": "2023_filtrado.csv",
    "2024": "2024_filtrado.csv",
    "2025": "2025_filtrado.csv"
}

# Diccionario para guardar los dataframes ya limpios
filtrado_limpio = {}

# Abreviaturas de los meses
meses_abreviados = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                    'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Columnas que se eliminarán si existen
columnas_a_eliminar = ["Cód. SS/SEREMI", "Cód. Estab.", "Cód. Nivel Cuidado", "Acum", "Año"]

# Procesamiento de los archivos
for año, archivo in archivos_filtrados.items():
    # Cargar CSV
    df = pd.read_csv(archivo)

    # Eliminar filas irrelevantes
    df = df[~df["Nombre Nivel Cuidado"].isin([
      "330 - Area Pensionado",
      "330 - Área Pensionado",
      "401 - Área Médica Adulto Cuidados Básicos",
      "402 - Área Médica Adulto Cuidados Medios",
      "403 - Área Médico-Quirúrgico Cuidados Básicos",
      "404 - Área Médico-Quirúrgico Cuidados Medios",
      "405 - Área Cuidados Intensivos Adultos",
      "406 - Área Cuidados Intermedios Adultos",
      "407 - Área Médica Pediátrica Cuidados Básicos",
      "408 - Área Médica Pediátrica Cuidados Medios",
      "409 - Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "410 - Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "411 - Área Cuidados Intensivos Pediátricos",
      "412 - Área Cuidados Intermedios Pediátricos",
      "413 - Área Neonatología Cuidados Básicos",
      "414 - Área Neonatología Cuidados Intensivos",
      "415 - Área Neonatología Cuidados Intermedios",
      "416 - Área Obstetricia",
      "418 - Área Psiquiatría Adulto Corta estadía",
      "419 - Área Psiquiatría Adulto Mediana estadía",
      "420 - Área Psiquiatría Adulto Larga estadía",
      "421 - Área Psiquiatría Infanto-adolescente corta estadía",
      "427 - Área Sociosanitaria Adulto",
      "428 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Adulto",
      "429 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Infanto Adolescente",
      "Área Cuidados Intensivos Adultos",
      "Área Cuidados Intensivos Pediátricos",
      "Área Cuidados Intermedios Adultos",
      "Área Cuidados Intermedios Pediátricos",
      "Área Médica Adulto Cuidados Básicos",
      "Área Médica Adulto Cuidados Medios",
      "Área Médica Pediátrica Cuidados Básicos",
      "Área Médica Pediátrica Cuidados Medios",
      "Área Médico-Quirúrgico Cuidados Básicos",
      "Área Médico-Quirúrgico Cuidados Medios",
      "Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "Área Neonatología Cuidados Básicos",
      "Área Neonatología Cuidados Intensivos",
      "Área Neonatología Cuidados Intermedios",
      "Área Obstetricia",
      "Área Pensionado",
      "Área Psiquiatría Adulto Corta estadía",
      "Área Psiquiatría Adulto Larga estadía",
      "Área Psiquiatría Adulto Mediana estadía",
      "Área Psiquiatría Infanto-adolescente corta estadía"
    ]
    )].copy()

    # Eliminar columnas innecesarias
    df = df.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas clave
    df = df.rename(columns={
        "Nombre SS/SEREMI": "SEREMI",
        "Nombre Nivel Cuidado": "Area"
    })

    # Renombrar columnas de meses abreviados por fecha
    nuevo_nombre_columnas = {}
    año_int = int(año)

    for i, mes_abrev in enumerate(meses_abreviados, start=1):
        if mes_abrev in df.columns:
            fecha = datetime(año_int, i, 1).strftime('%Y-%m-%d')
            nuevo_nombre_columnas[mes_abrev] = fecha

    df = df.rename(columns=nuevo_nombre_columnas)

    # Guardar DataFrame limpio
    filtrado_limpio[año] = df

# Ejemplo: verificar que todo cargó bien
filtrado_limpio["2020"].head(50)

,SEREMI,Nombre Establecimiento,Area,Glosa,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01
0,Datos País,Datos País,Datos País,Dias Cama Disponibles,800179.00,745007.00,789700.00,745965.00,802598.00,795021.00,832573.00,827810.00,779335.00,805056.00,776759.00,793770.00
1,Datos País,Datos País,Datos País,Dias Cama Ocupados,627256.00,577307.00,540914.00,412450.00,477211.00,498858.00,545163.00,557324.00,540777.00,567446.00,559618.00,552408.00
2,Datos País,Datos País,Datos País,Dias de Estada,627876.00,557377.00,602302.00,406231.00,439175.00,503875.00,548854.00,515769.00,527973.00,554252.00,532859.00,589847.00
3,Datos País,Datos País,Datos País,Promedio Cama Disponibles,25812.23,25689.90,25474.19,24865.50,25890.26,26500.70,26857.19,26703.55,25977.83,25969.55,25891.97,25605.48
4,Datos País,Datos País,Datos País,Numero de Egresos,89177.00,80451.00,79438.00,54961.00,58717.00,59720.00,65544.00,66102.00,67836.00,71445.00,70530.00,72361.00
5,Datos País,Datos País,Datos País,Egresos Fallecidos,2605.00,2261.00,2383.00,2246.00,3222.00,4739.00,3864.00,3381.00,3026.00,2924.00,2565.00,2799.00
6,Arica,Datos Servicio Salud,Datos Servicio Salud,Dias Cama Disponibles,9697.00,9066.00,9608.00,9660.00,9688.00,10159.00,11000.00,11674.00,11427.00,11426.00,11395.00,11337.00
7,Arica,Datos Servicio Salud,Datos Servicio Salud,Dias Cama Ocupados,8977.00,8535.00,8042.00,5209.00,6357.00,6985.00,8342.00,8989.00,8223.00,8957.00,8602.00,8269.00
8,Arica,Datos Servicio Salud,Datos Servicio Salud,Dias de Estada,8196.00,7182.00,9624.00,4785.00,5341.00,5879.00,7435.00,8042.00,7151.00,8490.00,7162.00,8778.00
9,Arica,Datos Servicio Salud,Datos Servicio Salud,Promedio Cama Disponibles,312.81,312.62,309.94,322.00,312.52,338.63,354.84,376.58,380.90,368.58,379.83,365.71


### Función para unión de datos:

In [7]:
def extraer_info_hospital_area_todo_el_año(hospital_nombre, area_nombre):
    registros = []

    for año, df in filtrado_limpio.items():
        df_seremi = seremi_data[año].copy()

        # Aseguramos que 'Mes' esté en formato datetime
        df_seremi['Mes'] = pd.to_datetime(df_seremi['Mes'], errors='coerce')
        df_seremi.dropna(subset=['Mes'], inplace=True)
        df_seremi['Mes'] = df_seremi['Mes'].dt.to_period("M").dt.to_timestamp()

        columnas_mes = [col for col in df.columns if col.startswith("20")]
        for col in columnas_mes:
            # Filtro hospitalario por hospital, área y mes
            df_tmp = df[
                (df["Nombre Establecimiento"] == hospital_nombre) &
                (df["Area"] == area_nombre)
            ][["SEREMI", "Nombre Establecimiento", "Area", "Glosa", col]].copy()

            if df_tmp.empty:
                continue

            df_tmp = df_tmp.rename(columns={col: "Valor"})
            df_tmp["Fecha"] = pd.to_datetime(col)

            # Filtro climático por seremi y mes
            clima_mes = df_seremi[df_seremi["Mes"] == df_tmp["Fecha"].iloc[0]].copy()
            df_merge = df_tmp.merge(clima_mes, on=["SEREMI"], how="left")

            registros.append(df_merge)

    if registros:
        return pd.concat(registros, ignore_index=True).sort_values("Fecha")
    else:
        print("⚠️ No se encontraron datos para ese hospital y área.")
        return pd.DataFrame()

# Ejemplo de prueba
ejemplo_df = extraer_info_hospital_area_todo_el_año(
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

dataframe=ejemplo_df

dataframe.head()

,SEREMI,Nombre Establecimiento,Area,Glosa,Valor,Fecha,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),Dirección del Viento
0,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias Cama Disponibles,3469.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
1,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias Cama Ocupados,3135.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
2,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias de Estada,2879.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
3,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Promedio Cama Disponibles,111.9,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
4,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Numero de Egresos,332.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW


### Preparación de datos pre modelación:

In [8]:
def preparar_dataset_climatico_avanzado(df):
    # Pivotear la tabla para tener una fila por fecha y cada glosa como columna
    df_pivot = df.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    # Comprobamos que estén todas las columnas hospitalarias necesarias
    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        print("❌ Datos insuficientes para generar el modelo.")
        return None

    # Extraemos variables climáticas (asumimos que son iguales por cada fecha para cada hospital)
    clima = df.drop_duplicates(subset='Fecha')[[
        'Fecha', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Dirección del Viento'
    ]]

    # Merge de hospital + clima
    df_pivot = pd.merge(df_pivot, clima, on='Fecha', how='left')

    # Codificar dirección del viento
    df_pivot = pd.get_dummies(df_pivot, columns=['Dirección del Viento'], prefix='Viento')

    # Orden cronológico
    df_pivot = df_pivot.sort_values('Fecha')
    
    # Variables de tiempo
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # Lags
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['lag_3'] = df_pivot['Dias Cama Disponibles'].shift(3)

    # Media móvil
    df_pivot['media_movil_3'] = df_pivot['Dias Cama Disponibles'].rolling(window=3).mean()

    # Diferencias hospitalarias
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    # Diferencia térmica
    df_pivot['Diferencia Térmica'] = df_pivot['Temperatura Máxima'] - df_pivot['Temperatura Mínima']

    # Tendencias climáticas
    df_pivot['temp_max_movil'] = df_pivot['Temperatura Máxima'].rolling(window=3).mean()
    df_pivot['precipitacion_movil'] = df_pivot['Precipitaciones (suma)'].rolling(window=3).mean()

    # Estacionalidad
    df_pivot['same_month_last_year'] = df_pivot['Dias Cama Disponibles'].shift(12)
    df_pivot['hist_avg_mes'] = df_pivot.groupby('Mes')['Dias Cama Disponibles'].transform('mean')

    # Interacciones cruzadas
    df_pivot['interaccion_ocupacion_temp'] = df_pivot['porcentaje_ocupacion'] * df_pivot['Temperatura Máxima']
    df_pivot['interaccion_precipitacion_disp'] = df_pivot['Precipitaciones (suma)'] * df_pivot['Dias Cama Disponibles']

    return df_pivot.dropna().copy()



### Automatización de extración de datos y preparación de dataset - MODELO XGBOOST:

Estamos prediciendo aproximadamente el 20% de los meses más recientes del historial disponible para ese hospital y área. Por ejemplo:

Si el hospital tiene datos desde 2014 hasta 2025, son 12 años × 12 meses = 144 - 10 (meses faltantes del 2025) -> 134 meses totales.

El 20% de 134 son aprox 27 meses -> estamos prediciendo los últimos 27 meses, desde fines de 2022 hasta marzo de 2025 (en nuestro caso actual).

In [9]:
def pipeline_final_mae_reducido(hospital, area):
    print(f"🔍 Extrayendo datos para: {hospital} | {area}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital, area)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Selección de features
    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # División temporal
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    print("🔧 Ajustando modelo con GridSearchCV...")
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    model = XGBRegressor(n_jobs=1, random_state=42)
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=3),
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    mejor_modelo = grid_search.best_estimator_
    print("📌 Mejores parámetros encontrados:", grid_search.best_params_)

    print("📊 Evaluando sobre el 20% final de los datos...")

    # Predicción
    y_pred = mejor_modelo.predict(X_test)

    # Cálculo de métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    # Resultado detallado con valores reales y predichos
    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,                  # 🔹 Valores reales observados
        'Predicho': y_pred,                     # 🔹 Valores predichos por el modelo
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    # Reporte resumen
    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados



### Ejemplo de uso - MODELO XGBOOST:

In [10]:
resultados_final = pipeline_final_mae_reducido(
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

display(resultados_final)

🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 56.44
✅ RMSE (camas): 78.82
📉 Error promedio porcentual: 2.65%


,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%)
0,2023-07-01,2325.0,2275.050049,49.949951,2.148385
1,2023-08-01,2325.0,2280.936523,44.063477,1.895203
2,2023-09-01,2252.0,2310.750488,58.750488,2.608814
3,2023-10-01,2334.0,2330.941162,3.058838,0.131056
4,2023-11-01,2252.0,2313.443604,61.443604,2.728402
5,2023-12-01,2144.0,2099.135254,44.864746,2.092572
6,2024-01-01,2146.0,2104.379639,41.620361,1.939439
7,2024-02-01,2035.0,2025.235718,9.764282,0.479817
8,2024-03-01,2458.0,2339.992920,118.007080,4.800939
9,2024-04-01,2400.0,2326.737061,73.262939,3.052622


### Ejemplo de uso extendido a mas hospitales - MODELO XGBOOST:

In [11]:
# 1. Función para extraer todos los pares únicos (hospital, área)
def extraer_hospitales_y_areas_validos(filtrado_limpio):
    pares_hospital_area = set()
    for df in filtrado_limpio.values():
        if "Nombre Establecimiento" in df.columns and "Area" in df.columns:
            pares = df[["Nombre Establecimiento", "Area"]].dropna().drop_duplicates()
            for _, row in pares.iterrows():
                pares_hospital_area.add((row["Nombre Establecimiento"], row["Area"]))
    return sorted(list(pares_hospital_area))

# 2. Obtener los pares válidos y limitar a los primeros 30
hospitales_areas_validos = extraer_hospitales_y_areas_validos(filtrado_limpio)
pares_a_evaluar = hospitales_areas_validos[:30]

# 3. Evaluar modelo para cada hospital
resultados_todos = {}

for hospital, area in pares_a_evaluar:
    try:
        print("\n==========================")
        print(f"🏥 Evaluando: {hospital} | Área: {area}")
        print("==========================")
        resultados = pipeline_final_mae_reducido(hospital, area)
        if resultados is not None:
            resultados_todos[f"{hospital} | {area}"] = resultados
    except Exception as e:
        print(f"❌ Error en {hospital} con área {area}: {e}")
        continue

# 4. Crear tabla resumen
tabla_resumen = []

for nombre_completo, resultados in resultados_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        hospital, area = nombre_completo.split(" | ", 1)
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        tabla_resumen.append({
            "Hospital": hospital,
            "Área": area,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

# 5. Mostrar tabla ordenada
df_tabla_resumen = pd.DataFrame(tabla_resumen).sort_values("MAE (camas)").reset_index(drop=True)
display(df_tabla_resumen)




🏥 Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 330 - Área Pensionado
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 330 - Área Pensionado
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
❌ No se pudo preparar el dataset.

🏥 Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 401 - Área Médica Adulto Cuidados Básicos
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 190.52
✅ RMSE (camas): 238.21
📉 Error promedio porcentual: 9.28%

🏥 Evaluando: Complejo Hospitalario San 

,Hospital,Área,MAE (camas),Error Porcentual Promedio (%)
0,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intermedios,0.51,0.06
1,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intensivos Adultos,0.76,0.23
2,Hospital Adalberto Steeger (Talagante),405 - Área Cuidados Intensivos Adultos,1.05,0.60
3,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intermedios Adultos,1.36,0.14
4,"Complejo Hospitalario San José (Santiago, Inde...",Área Pensionado,2.43,0.40
5,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intensivos,4.35,1.08
6,"Complejo Hospitalario San José (Santiago, Inde...",413 - Área Neonatología Cuidados Básicos,4.50,0.92
7,Hospital Adalberto Steeger (Talagante),407 - Área Médica Pediátrica Cuidados Básicos,6.44,3.80
8,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Básicos,7.63,1.80
9,"Complejo Hospitalario San José (Santiago, Inde...",414 - Área Neonatología Cuidados Intensivos,8.10,2.40


### Listado y cantidad de hospitales con los que trabaja el modelo:

In [12]:
def obtener_lista_hospitales(filtrado_limpio):
    hospitales = set()
    
    for año, df in filtrado_limpio.items():
        if "Nombre Establecimiento" in df.columns:
            hospitales.update(df["Nombre Establecimiento"].dropna().unique())
    
    return sorted(list(hospitales))


In [13]:
lista_hospitales = obtener_lista_hospitales(filtrado_limpio)
print(f"Se encontraron {len(lista_hospitales)} hospitales.")
for h in lista_hospitales[:10]:  # muestra los primeros 10 como ejemplo
    print("🏥", h)


Se encontraron 92 hospitales.
🏥 Complejo Hospitalario San José (Santiago, Independencia)
🏥 Hospital Adalberto Steeger (Talagante)
🏥 Hospital Adriana Cousiño (Quintero)
🏥 Hospital Barros Luco Trudeau (Santiago, San Miguel)
🏥 Hospital Carlos Van Buren (Valparaíso)
🏥 Hospital Centro Geriátrico Paz de la Tarde (Limache)
🏥 Hospital Clorinda Avello (Santa Juana)
🏥 Hospital Clínico Herminda Martín (Chillán)
🏥 Hospital Comunitario de Salud Familiar Pedro Morales Campos (Yungay)
🏥 Hospital Comunitario de Salud Familiar de Bulnes


# 
92 hospitales comunes entre 2014 y 2025

### Listado y cantidad de Áreas Médicas con los que trabaja el modelo:

In [14]:
def obtener_lista_areas(filtrado_limpio):
    areas = set()
    
    for año, df in filtrado_limpio.items():
        if "Area" in df.columns:
            areas.update(df["Area"].dropna().unique())
    
    return sorted(list(areas))


In [15]:
lista_areas = obtener_lista_areas(filtrado_limpio)
print(f"Se encontraron {len(lista_areas)} áreas médicas.")
for a in lista_areas[:48]:  # muestra las primeras 10 como ejemplo
    print("📌", a)


Se encontraron 47 áreas médicas.
📌 330 - Area Pensionado
📌 330 - Área Pensionado
📌 401 - Área Médica Adulto Cuidados Básicos
📌 402 - Área Médica Adulto Cuidados Medios
📌 403 - Área Médico-Quirúrgico Cuidados Básicos
📌 404 - Área Médico-Quirúrgico Cuidados Medios
📌 405 - Área Cuidados Intensivos Adultos
📌 406 - Área Cuidados Intermedios Adultos
📌 407 - Área Médica Pediátrica Cuidados Básicos
📌 408 - Área Médica Pediátrica Cuidados Medios
📌 409 - Área Médico-Quirúrgico Pediátrica Cuidados Básicos
📌 410 - Área Médico-Quirúrgico Pediátrica Cuidados Medios
📌 411 - Área Cuidados Intensivos Pediátricos
📌 412 - Área Cuidados Intermedios Pediátricos
📌 413 - Área Neonatología Cuidados Básicos
📌 414 - Área Neonatología Cuidados Intensivos
📌 415 - Área Neonatología Cuidados Intermedios
📌 416 - Área Obstetricia
📌 418 - Área Psiquiatría Adulto Corta estadía
📌 419 - Área Psiquiatría Adulto Mediana estadía
📌 420 - Área Psiquiatría Adulto Larga estadía
📌 421 - Área Psiquiatría Infanto-adolescente corta

### XGBOOST para predicción futura: (MODO ENTRENAMIENTO)

In [32]:
def predecir_camas_mes_futuro(hospital, area, modo="entrenamiento", mes_futuro=None):
    print(f"🔍 Extrayendo datos para: {hospital} | {area}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital, area)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Selección de features
    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    if modo == "entrenamiento":
        # División entrenamiento/test
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

        print("🔧 Ajustando modelo con GridSearchCV...")
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.05, 0.1]
        }

        model = XGBRegressor(n_jobs=1, random_state=42)
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='neg_mean_absolute_error',
            cv=TimeSeriesSplit(n_splits=3),
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        mejor_modelo = grid_search.best_estimator_

        print("📌 Mejores parámetros encontrados:", grid_search.best_params_)
        print("📊 Evaluando sobre el 20% final de los datos...")

        y_pred = mejor_modelo.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

        print(f"✅ MAE (camas): {mae:.2f}")
        print(f"✅ RMSE (camas): {rmse:.2f}")
        print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

        resultados = pd.DataFrame({
            'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
            'Real': y_test.values,
            'Predicho': y_pred,
            'Error Absoluto': np.abs(y_test.values - y_pred),
            'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
        })

        return resultados

    else:
        print("⚠️ Modo no reconocido o no implementado aún.")
        return None



In [37]:
# Crear resultados para entrenamiento iterando hospitales
resultados_entrenamiento_todos = []

for hospital in lista_hospitales:

    if not lista_areas:
        continue

    area_random = random.choice(lista_areas)
    try:
        print("\n==========================")
        print(f"🏥 Evaluando entrenamiento: {hospital} | Área aleatoria: {area_random}")
        print("==========================")

        resultado_entrenamiento = predecir_camas_mes_futuro(
            hospital=hospital,
            area=area_random,
            modo="entrenamiento"
        )

        if resultado_entrenamiento is not None:
            resultado_entrenamiento["Hospital"] = hospital
            resultado_entrenamiento["Área"] = area_random
            resultados_entrenamiento_todos.append(resultado_entrenamiento)

    except Exception as e:
        print(f"❌ Error en {hospital} | Área: {area_random} -> {e}")
        continue





🏥 Evaluando entrenamiento: Complejo Hospitalario San José (Santiago, Independencia) | Área aleatoria: Área Médico-Quirúrgico Cuidados Medios
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | Área Médico-Quirúrgico Cuidados Medios
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 102.61
✅ RMSE (camas): 112.80
📉 Error promedio porcentual: 3.30%

🏥 Evaluando entrenamiento: Hospital Adalberto Steeger (Talagante) | Área aleatoria: 401 - Área Médica Adulto Cuidados Básicos
🔍 Extrayendo datos para: Hospital Adalberto Steeger (Talagante) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rat

,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%),Hospital,Área
0,2015-10-01,3363.0,3324.772949,38.227051,1.136695,"Complejo Hospitalario San José (Santiago, Inde...",Área Médico-Quirúrgico Cuidados Medios
1,2015-11-01,2830.0,2681.626953,148.373047,5.242864,"Complejo Hospitalario San José (Santiago, Inde...",Área Médico-Quirúrgico Cuidados Medios
2,2015-12-01,3446.0,3324.772949,121.227051,3.517906,"Complejo Hospitalario San José (Santiago, Inde...",Área Médico-Quirúrgico Cuidados Medios
3,2020-09-01,678.0,674.895386,3.104614,0.457908,Hospital Adalberto Steeger (Talagante),401 - Área Médica Adulto Cuidados Básicos
4,2020-10-01,711.0,706.427002,4.572998,0.643178,Hospital Adalberto Steeger (Talagante),401 - Área Médica Adulto Cuidados Básicos
...,...,...,...,...,...,...,...
242,2024-10-01,2954.0,2268.814453,685.185547,23.195178,Hospital de Villarrica,Datos Establecimiento
243,2024-11-01,2774.0,2141.202637,632.797363,22.811729,Hospital de Villarrica,Datos Establecimiento
244,2024-12-01,3008.0,2267.984619,740.015381,24.601575,Hospital de Villarrica,Datos Establecimiento
245,2025-01-01,3012.0,2237.214844,774.785156,25.723279,Hospital de Villarrica,Datos Establecimiento


In [40]:
# Unir todos los resultados
df_entrenamiento_resultados = pd.concat(resultados_entrenamiento_todos, ignore_index=True)
dataframe=df_entrenamiento_resultados

dataframe_ordenado = dataframe.sort_values(by="Error Porcentual (%)", ascending=False)


dataframe_ordenado.head(50)

,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%),Hospital,Área
201,2025-02-01,172.0,383.393219,211.393219,122.903034,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
193,2024-06-01,186.0,398.997833,212.997833,114.514964,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
199,2024-12-01,191.0,398.109680,207.109680,108.434388,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
200,2025-01-01,195.0,399.901489,204.901489,105.077687,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
198,2024-11-01,192.0,391.026093,199.026093,103.659423,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
192,2024-05-01,193.0,391.120880,198.120880,102.653306,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
189,2024-02-01,203.0,392.245361,189.245361,93.224316,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
191,2024-04-01,207.0,389.709839,182.709839,88.265623,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
188,2024-01-01,208.0,391.263000,183.263000,88.107212,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos
190,2024-03-01,219.0,402.801697,183.801697,83.927715,Hospital de Salamanca,401 - Área Médica Adulto Cuidados Básicos


In [41]:
promedio = dataframe["Error Porcentual (%)"].mean()

print("El promedio de la columna es:", promedio)


El promedio de la columna es: 9.998263622142609
